# Secret Santa Explained

Step by step, we develop a secure MPC protocol for the [Secret Santa](https://en.wikipedia.org/wiki/Secret_Santa) problem. Traditionally, a group of family and friends gathers to put all their names in a hat and then randomly draw names from the hat. If someone draws their own name, they have to start all over again. 

Mathematically, the Secret Santa problem is about generating so-called [derangements](https://en.wikipedia.org/wiki/Derangement), which are permutations without fixed points. A permutation is a one-to-one mapping on a set of numbers, and a fixed-point is a number that is mapped to itself. 

We present an MPyC program (a Python program using the `mpyc` package) for generating uniformly random derangements. 
In this notebook, the MPyC program is run by a single party only. However, this very same MPyC program can be run between multiple parties to generate secret random derangements. These random derangements will remain secret *forever* unless a majority of these parties collude.

## MPyC setup

To get started, we simply import the MPyC runtime `mpc` at the start of our program.

In [1]:
from mpyc.runtime import mpc

A derangement of length $n$ is a permutation of the numbers $0, 1, ..., n-1$ without fixed points. 

Represented as Python lists, the 12 smallest derangements are:

|$n$| <p align="justify">length-$n$ derangements|
|---|:----------------------|
| 2 | <p align="left">[1,0]  |
| 3 | <p align="left">[1,2,0], [2,0,1]  |
| 4 | <p align="left">[1,0,3,2], [1,2,3,0], [1,3,0,2], [2,0,3,1], [2,3,0,1], [2,3,1,0], [3,0,1,2], [3,2,0,1], [3,2,1,0]  |

To represent *secret* derangements, we will use a secure MPyC type of integers. For simplicity, we choose 32-bit (default) secure integers.

In [2]:
secint = mpc.SecInt() # 32-bit secure MPyC integers

## Random derangements from random permutations

To generate a uniformly random derangement, we basically proceed in the traditional way: we draw all the numbers from a hat in order and start all over if there are any fixed-points. A permutation $a$, viewed as a sequence of length $n$, is a derangement exactly when $\forall_{0\leq i<n} \ a_i \neq i$.

We represent a secret derangement by a list `a` with elements of type `secint`. The idea is to first generate a random permutation and then check if this permutation happens to be free of fixed points. If any fixed points are found, we have to start all over again.

To use this idea for a secure computation, we have to find a way to check if `a` is free of fixed points without leaking any further information about `a`. The elements of `a` are all secret and should remain so. The following property tells us how we can find this single bit of information on `a` by a simple computation over secure integers:

$$\forall_{0\leq i<n} \ a_i \neq i \ \ \ \Leftrightarrow \ \ \ \prod_{i=0}^{n-1} \ (a_i - i) \neq 0$$

The leads to the following MPyC code for function `random_derangement`:

In [3]:
@mpc.coroutine                                      # turn coroutine into an MPyC coroutine
async def random_derangement(n):      
    await mpc.returnType(secint, n)                 # set the return type of this MPyC coroutine
    a = random_permutation(n)
    t = mpc.prod([a[i] - i for i in range(n)])      # securely compute product of all differences a[i] - i
    if await mpc.is_zero_public(t):                 # publicly test whether t is equal to zero
        return random_derangement(n)                # recurse if t is zero
    else:
        return a                                    # done if t is non-zero

Function `random_derangement` uses these four functions from the `mpc` runtime:

1. `mpc.prod(x)` to securely compute the product of all elements in `x`.

2. `mpc.is_zero_public(a)` to test securely whether `a` is equal to 0, revealing only the outcome publicly.

3. Decorator `mpc.coroutine(f)` to turn coroutine `f` into an MPC coroutine.

4. `mpc.returnType(rettype)` to define the return type of an MPC coroutine.

We have defined function `random_derangement` as a coroutine because from its body we want to call function `mpc.is_zero_public`, which is a couroutine as well. When calling a coroutine explicitly, we have to wait for its completion using the `await` keyword.

## Random permutations from random unit vectors

The [Fisher-Yates shuffle (or, Knuth shuffle)](https://en.wikipedia.org/wiki/Fisher–Yates_shuffle) is a classic algorithm for generating permutations uniformly at random. The Python program is very simple:

```
    a = list(range(n))
    for i in range(n - 1):
        r = random.randrange(i, n)
        a[i], a[r] = a[r], a[i]
```

To implement the shuffle securely, however, we have to hide which elements of `a` are swapped in each loop iteration. To hide this properly we have to hide both the random index `r` and which elements of `a` are modified by the swap. 


In [4]:
def random_permutation(n):
    a = [secint(i) for i in range(n)]               # initialize a to identity permutation
    for i in range(n - 1):
        x = random_unit_vector(n - i)               # x = ([0] * r) + [1] + ([0] * (n - i - 1 -r), random r in range(n - i)
        a_x = mpc.in_prod(a[i - n:], x)             # securly compute a[r] ...
        d = mpc.scalar_mul(a[i] - a_x, x)           # ...
        a[i] = a_x
        for j in range(n - i):
            a[i + j] += d[j]
    return a

The function `random_permutation` is defined as a plain Python function because we do not need to wait for any results explicitly. Two further functions from the `mpc` runtime are used:

1. `mpc.in_prod(x, y)` to securely compute the dot product of `x` and `y`.

2. `mpc.scalar_mul(a, x)` to securely compute the product of `a` with each element of `x`.

## Random unit vectors

Final step is to generate unit vectors of a given length uniformly at random. A unit vector of length $n$ is any bit vector with exactly one 1 bit and $n-1$ 0 bits. 

Our algorithm for generating random unit vectors will be recursive. The base case is $n=1$, ...


In [5]:
@mpc.coroutine
async def random_unit_vector(n):
    await mpc.returnType(secint, n)
    if n == 1: 
        return [secint(1)]
    b = mpc.random_bit(secint)
    x = random_unit_vector((n + 1) // 2)
    if n % 2 == 0:
        y = mpc.scalar_mul(b, x)
        return y + mpc.vector_sub(x, y)
    elif await mpc.is_zero_public(b * x[0] - 1):
        return random_unit_vector(n)
    else:
        y = mpc.scalar_mul(b, x[1:])
        return y + mpc.vector_sub(x[1:], y) + x[:1]

## Test drive

Let's now check what the results look like. We check the first few cases for each function.

In [6]:
m = 7

In [7]:
for n in range(2, m + 1):
    s = mpc.run(mpc.output(random_unit_vector(n)))
    print("{0:2} {1}".format(n, s))

 2 [1, 0]
 3 [1, 0, 0]
 4 [0, 0, 1, 0]
 5 [0, 0, 0, 1, 0]
 6 [0, 1, 0, 0, 0, 0]
 7 [0, 0, 0, 0, 0, 0, 1]


In [8]:
for n in range(2, m + 1):
    s = mpc.run(mpc.output(random_permutation(n)))
    print("{0:2} {1}".format(n, s))

 2 [1, 0]
 3 [1, 0, 2]
 4 [3, 1, 2, 0]
 5 [2, 4, 1, 3, 0]
 6 [2, 1, 3, 5, 0, 4]
 7 [3, 4, 0, 5, 6, 1, 2]


In [9]:
for n in range(2, m + 1):
    s = mpc.run(mpc.output(random_derangement(n)))
    print("{0:2} {1}".format(n, s))

 2 [1, 0]
 3 [2, 0, 1]
 4 [3, 2, 1, 0]
 5 [3, 4, 1, 2, 0]
 6 [3, 4, 0, 1, 5, 2]
 7 [6, 2, 3, 0, 5, 1, 4]
